This notebook prepares the documentation on Newton's method, deflation, and multiplicity.

# Newton's method, Deflation and Multiplicity

In [1]:
from phcpy.solutions import make_solution
from phcpy.deflation import double_newton_step
from phcpy.deflation import double_double_newton_step
from phcpy.deflation import quad_double_newton_step
from phcpy.deflation import double_deflate
from phcpy.deflation import double_multiplicity

PHCv2.4.88 released 2023-12-26 works!


## 1. Newton's method

In [2]:
pols = ['x^2 + 4*y^2 - 4;' '2*y^2 - x;']

At a regular solution, Newton's method doubles the accuracy in each step. 

In [3]:
sols = [make_solution(['x', 'y'], [1.23, -0.786])]
print(sols[0])

t : 0.000000000000000E+00 0.000000000000000E+00
m : 1
the solution for t :
 x : 1.230000000000000E+00  0.0
 y : -7.860000000000000E-01  0.0
== err : 0.000E+00 = rco : 1.000E+00 = res : 0.000E+00 =


In [4]:
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 0
the solution for t :
 x :  1.23085880440470E+00   0.00000000000000E+00
 y : -7.88195187844216E-01   0.00000000000000E+00
== err :  2.195E-03 = rco :  0.000E+00 = res :  2.001E-05 =


Observe the values for ``err`` and ``res``.

In [5]:
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 0
the solution for t :
 x :  1.23085772919640E+00   0.00000000000000E+00
 y : -7.88192433754122E-01   0.00000000000000E+00
== err :  2.754E-06 = rco :  0.000E+00 = res :  3.150E-11 =


In [6]:
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 0
the solution for t :
 x :  1.23085772919471E+00   0.00000000000000E+00
 y : -7.88192433749787E-01   0.00000000000000E+00
== err :  4.335E-12 = rco :  0.000E+00 = res :  2.220E-16 =


In [7]:
sols = double_double_newton_step(pols, sols)
print(sols[0])

t : 0.00000000000000000000000000000000E+00      0.00000000000000000000000000000000E+00    
m : 0
the solution for t :
 x : 1.23085772919471000000000000000000E+00      0.00000000000000000000000000000000E+00    
 y : -7.88192433749787000000000000000007E-01     0.00000000000000000000000000000000E+00    
== err :  4.335E-12 = rco :  0.000E+00 = res :  2.220E-16 =


In [8]:
sols = double_double_newton_step(pols, sols)
print(sols[0])

t : 0.00000000000000000000000000000000E+00      0.00000000000000000000000000000000E+00    
m : 0
the solution for t :
 x : 1.23085772919470999999999999999999E+00      0.00000000000000000000000000000000E+00    
 y : -7.88192433749787000000000000000007E-01     0.00000000000000000000000000000000E+00    
== err :  4.335E-12 = rco :  0.000E+00 = res :  2.220E-16 =


In [9]:
sols = quad_double_newton_step(pols, sols)
print(sols[0])

t : 0.0000000000000000000000000000000000000000000000000000000000000000E+00      0.0000000000000000000000000000000000000000000000000000000000000000E+00    
m : 0
the solution for t :
 x : 1.2308577291947099999999999999999900000000000000000000000000000000E+00      0.0000000000000000000000000000000000000000000000000000000000000000E+00    
 y : -7.8819243374978700000000000000000700000000000000000000000000000001E-01     0.0000000000000000000000000000000000000000000000000000000000000000E+00    
== err :  4.335E-12 = rco :  0.000E+00 = res :  2.220E-16 =


## 2. Deflation

At an isolated singular solution, *deflation* is a method to restore the quadratic convergence of Newton's method.

In [10]:
pols = ['(29/16)*x^3 - 2*x*y;', 'x^2 - y;']

In [11]:
sols = [make_solution(['x', 'y'],[1.0e-6, 1.0e-6])]
print(sols[0])

t : 0.000000000000000E+00 0.000000000000000E+00
m : 1
the solution for t :
 x : 1.000000000000000E-06  0.0
 y : 1.000000000000000E-06  0.0
== err : 0.000E+00 = rco : 1.000E+00 = res : 0.000E+00 =


In [12]:
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 0
the solution for t :
 x :  9.99999906191101E-07   0.00000000000000E+00
 y :  9.99999812409806E-13   0.00000000000000E+00
== err :  1.000E-06 = rco :  5.625E-13 = res :  1.875E-19 =


In [13]:
sols = double_newton_step(pols, sols)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 0
the solution for t :
 x :  6.66666604160106E-07   0.00000000000000E+00
 y :  3.33333270859482E-13   0.00000000000000E+00
== err :  3.333E-07 = rco :  2.778E-14 = res :  1.111E-13 =


In [14]:
solsd = double_deflate(pols, sols)
print(solsd[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 x :  2.11572444293177E-22  -2.71418638493962E-25
 y :  7.81101038268534E-24   7.59985933345404E-23
== err :  1.346E-11 = rco :  1.512E-02 = res :  1.735E-16 =


Deflation also works on systems with more equations than unknowns.

In [15]:
pols = ['x^2;', 'x*y;', 'y^2;']

In [16]:
sols = [make_solution(['x', 'y'], [1.0e-6, 1.0e-6])]
print(sols[0])

t : 0.000000000000000E+00 0.000000000000000E+00
m : 1
the solution for t :
 x : 1.000000000000000E-06  0.0
 y : 1.000000000000000E-06  0.0
== err : 0.000E+00 = rco : 1.000E+00 = res : 0.000E+00 =


In [17]:
sols = double_deflate(pols, sols, tolrnk=1.0e-8)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 x :  1.25000000000000E-07   0.00000000000000E+00
 y :  1.25000000000000E-07   0.00000000000000E+00
== err :  1.250E-07 = rco :  8.165E-01 = res :  1.562E-14 =


In [18]:
sols = double_deflate(pols, sols, tolrnk=1.0e-4)
print(sols[0])

t :  0.00000000000000E+00   0.00000000000000E+00
m : 1
the solution for t :
 x :  2.93873587705572E-39   1.74487442700183E-39
 y :  5.87747175411144E-39  -6.24481373874340E-39
== err :  2.713E-23 = rco :  4.082E-01 = res :  2.260E-38 =


## 3. multiplicity structure

The multiplicity can be computed *locally* starting at the solution.

In [19]:
pols = [ 'x**2+y-3;', 'x+0.125*y**2-1.5;']

In [20]:
sol = make_solution(['x', 'y'], [1, 2])
print(sol)

t : 0.000000000000000E+00 0.000000000000000E+00
m : 1
the solution for t :
 x : 1.000000000000000E+00  0.0
 y : 2.000000000000000E+00  0.0
== err : 0.000E+00 = rco : 1.000E+00 = res : 0.000E+00 =


In [21]:
multiplicity, hilbert_function = double_multiplicity(pols, sol)

In [22]:
multiplicity

3

Thus, the multiplicity of ``(1, 2)`` as solution equals three.  This is confirmed by the Hilbert function.

In [23]:
hilbert_function

[1, 1, 1, 0, 0, 0]